In [ ]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from dateutil.parser import parse
import time
import os
import pickle

In [ ]:
# Anzahl der angezeigten Zeilen in JNP einstellen
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
# Pickl-Dateipfade vorbereiten:

data_path_wma = '/home/paul/python_projects/masterthesis/wma/data/wmm_data/'
data_path_weather = '/home/paul/python_projects/masterthesis/wma/data/wetter/'

pkl_file_wma = 'daten_wmm_all_prepared.p'
pkl_file_wetter_raw_dwn = 'downloaded/daten_wetter_all_raw.p'

In [ ]:
# Pickl-Dateien einlesen:

df_wma_prepared = pickle.load(open(data_path_wma + pkl_file_wma, 'rb'))
df_wetter_raw = pickle.load(open(data_path_weather + pkl_file_wetter_raw_dwn, 'rb'))

In [ ]:
df_wma_prepared.head()

In [ ]:
df_wetter_raw.head()

In [ ]:
# Kopie der Wetterdaten für die weitere Verarbeitung erstellen
df_wetter_1 = df_wetter_raw.copy()

# Kopie der Marathondaten mit notwendigen Spalten erstellen
df_wma_1 = df_wma_prepared[['Jahr','Ort','Datum','Datum_Startzeit_UTC']].copy()
df_wma_1.head()

In [ ]:
# Neue Spalte DATE_TIME aus DATE erzeugen.
df_wetter_1['DATE_TIME'] = df_wetter_1['DATE'].astype('datetime64')
# Uhrzeit aus der DATE Spalte entfernen, um später nur die relevanten Daten herauszufiltern
df_wetter_1['DATE'] = df_wetter_1['DATE_TIME'].dt.normalize()
df_wetter_1.head()

In [ ]:
# Mithilfe der WMA-Daten nur die relevanten Zeiträume der Wetterdaten filtern (mittels JOIN)
df_wetter_2 = pd.merge(left=df_wma_1, right=df_wetter_1, how='inner', left_on=['Ort','Datum'], right_on=['ORT','DATE'])
df_wetter_2.head()

In [29]:
# Nicht benötigte (doppelte) Spalten entfernen
df_wetter_3 = df_wetter_2.drop(columns=['ORT','DATE']).copy()
df_wetter_3.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0041,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 00:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"190,1,N,0041,1","01800,1,C,N","011265,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 00:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0041,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 01:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"190,1,N,0046,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 01:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0036,1","01800,1,C,N","011265,1,N,1","+0120,1","+0090,1","99999,9",2007-09-30 02:20:00


In [30]:
# Spalten hinzufügen die die Zeitspanne des Lauf repräsentieren
df_wetter_3['RANGE_DT_START'] = df_wetter_3['Datum_Startzeit_UTC'] 
df_wetter_3['RANGE_DT_END'] = None
df_wetter_3.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,DATE_TIME,RANGE_DT_START,RANGE_DT_END
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0041,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 00:20:00,2007-09-30 07:15:00,None
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"190,1,N,0041,1","01800,1,C,N","011265,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 00:50:00,2007-09-30 07:15:00,None
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0041,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 01:20:00,2007-09-30 07:15:00,None
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"190,1,N,0046,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 01:50:00,2007-09-30 07:15:00,None
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0036,1","01800,1,C,N","011265,1,N,1","+0120,1","+0090,1","99999,9",2007-09-30 02:20:00,2007-09-30 07:15:00,None


In [31]:
# Startzeitstpanne runden - 30 min
df_wetter_3['RANGE_DT_START'] = df_wetter_3['RANGE_DT_START'].dt.round('30min')

In [32]:
df_wetter_3.loc[(df_wetter_3.Ort == 'London') & (df_wetter_3.Jahr == '2019')].head(10)

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,DATE_TIME,RANGE_DT_START,RANGE_DT_END
41700,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"280,1,N,0082,1","01372,1,C,N","009999,1,9,9","+0100,1","+0020,1","99999,9",2019-04-28 00:20:00,2019-04-28 09:00:00,None
41701,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"270,1,N,0077,1","01402,1,C,N","009999,1,9,9","+0100,1","+0020,1","99999,9",2019-04-28 00:50:00,2019-04-28 09:00:00,None
41702,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"250,1,N,0072,1","01250,1,C,N","009999,1,9,9","+0090,1","+0020,1","99999,9",2019-04-28 01:20:00,2019-04-28 09:00:00,None
41703,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"260,1,N,0072,1","01128,1,C,N","009999,1,9,9","+0090,1","+0030,1","99999,9",2019-04-28 01:50:00,2019-04-28 09:00:00,None
41704,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"280,1,N,0082,1","00945,1,C,N","009999,1,9,9","+0080,1","+0040,1","99999,9",2019-04-28 02:20:00,2019-04-28 09:00:00,None
41705,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"280,1,V,0077,1","00975,1,C,N","009999,1,9,9","+0090,1","+0030,1","99999,9",2019-04-28 02:50:00,2019-04-28 09:00:00,None
41706,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"280,1,N,0057,1","00975,1,C,N","009999,1,9,9","+0090,1","+0040,1","99999,9",2019-04-28 03:20:00,2019-04-28 09:00:00,None
41707,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"280,1,V,0072,1","00884,1,C,N","009999,1,9,9","+0090,1","+0030,1","99999,9",2019-04-28 03:50:00,2019-04-28 09:00:00,None
41708,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"280,1,V,0067,1","00853,1,C,N","009999,1,9,9","+0090,1","+0040,1","99999,9",2019-04-28 04:20:00,2019-04-28 09:00:00,None
41709,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"290,1,N,0072,1","00792,1,C,N","009999,1,9,9","+0090,1","+0040,1","99999,9",2019-04-28 04:50:00,2019-04-28 09:00:00,None


In [34]:
# Startendzeitspanne berechnen: Es werden 3h zu der Startzeit hinzugerechnet, weil die Läufe zwischen 2 und 3 Stunden dauern (im Durchschnitt 2,5h)
df_wetter_3['RANGE_DT_END'] = df_wetter_3['RANGE_DT_START'] + dt.timedelta(hours=3)
df_wetter_3.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,DATE_TIME,RANGE_DT_START,RANGE_DT_END
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0041,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 00:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"190,1,N,0041,1","01800,1,C,N","011265,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 00:50:00,2007-09-30 07:00:00,2007-09-30 10:00:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0041,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 01:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"190,1,N,0046,1","99999,9,C,Y","009900,1,N,1","+0130,1","+0090,1","99999,9",2007-09-30 01:50:00,2007-09-30 07:00:00,2007-09-30 10:00:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0036,1","01800,1,C,N","011265,1,N,1","+0120,1","+0090,1","99999,9",2007-09-30 02:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00


In [ ]:
df_wetter_4 = df_wetter_3.copy()